#

In [214]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score

In [223]:
class NicheModelCommunityDetectionExperiment():
    def __init__(
        self,
        species=100, 
        connectance = 0.1,
        prop_edges_to_remove = 0.1,
        num_replicates = 64
        ):
        self.species = species
        self.connectance = connectance
        self.prop_edges_to_remove= prop_edges_to_remove
        self.num_replicates = num_replicates
   
    def clustering_labels_to_vector(self, clustering_labs):
        lab = np.zeros(self.species)
        for (ct,l) in enumerate(clustering_labs):
            for i in list(l):
                lab[i] = ct
        return lab
    
    def run_experiment(self):
        results  = []
        for r in range(self.num_replicates):
            M = self.make_foodweb()
            full_coms = nx.community.louvain_communities(M)
            full_network_labels = self.clustering_labels_to_vector(full_coms)
            full_Q = nx.community.modularity(M, full_coms)

            
            for s in range(self.num_replicates):
                S = self.remove_edges(M, self.prop_edges_to_remove)
                sampled_coms = nx.community.louvain_communities(S)
                sampled_labels = self.clustering_labels_to_vector(sampled_coms)
                sampled_Q = nx.community.modularity(S, sampled_coms)
                nmi = normalized_mutual_info_score(full_network_labels, sampled_labels)
                results.append({
                    "metaweb": r,
                    "num_species": self.species,
                    "prop_removed": self.prop_edges_to_remove,
                    "nmi": nmi,
                    "sampled_Q": sampled_Q,
                    "full_Q" : full_Q 
                })
        return results

    def remove_edges(self, G : nx.Graph, prop):
        original_edges = list(G.edges)
        np.random.shuffle(original_edges)
        n = int(np.floor(G.number_of_edges() * (1-prop)))
        new_edges = original_edges[0:n]
        return nx.Graph(new_edges)
    
    def make_foodweb(self):
        species, connectance = self.species, self.connectance
        beta = 1/(2*connectance)  - 1
        niche = np.random.uniform(size=species)
        niche_width = np.random.beta(1., beta, size=species)
        ranges = [niche_width[i]*niche[i] for i in range(len(niche))]
        
        centroids = [np.random.uniform(ranges[i]/2, niche[i]) for i in range(len(niche))]
        
        ranges[np.argmin(niche)] = 0
        edgelist = []
        for consumer in range(species):
            for resource in range(species):
                if niche[resource] < centroids[consumer] + 0.5*ranges[consumer]:
                    if niche[resource] > centroids[consumer] - 0.5*ranges[consumer]:
                        edgelist.append((consumer,resource))

        return nx.from_edgelist(edgelist)

In [224]:
results = []
for prop in np.linspace(0.05, 0.5, num=10):
    for sp in [25, 50, 100, 200]:
        experiment = NicheModelCommunityDetectionExperiment(
            species = sp,
            prop_edges_to_remove = prop
        )
        labs = experiment.run_experiment()
        results.append(labs)

In [241]:
df = pd.DataFrame(results[0])

for r in results[1:len(results)]:
    df = df.append(pd.DataFrame(r))

/tmp/ipykernel_298141/3297655543.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(r))
/tmp/ipykernel_298141/3297655543.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(r))
/tmp/ipykernel_298141/3297655543.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(r))
/tmp/ipykernel_298141/3297655543.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(r))
/tmp/ipykernel_298141/3297655543.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df

In [248]:
df.to_csv("niche.csv")